# CRIAÇÃO DA CLASSE DE EXTRAÇÃO DE FEATURES

Esse jupyter tem como objetivo implementar o maior número possível de funções para extração de features. Essas funções serão testadas para que depois possa ser criada uma classe.

In [1]:
import os
import librosa
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from IPython.display import Audio

## Tamanho do janelamento

É preciso analisar o dataset para verificar qual vai ser o tamanho do janelamento. Para garantir que o processamento seja o mais rápido possível, o tamanho da janela será o tamanho do menor áudio.

As células abaixo verificarão qual é o tamanho do menor arquivo do dataset.

In [ ]:
arrayDuracoes = []

diretorio = "/home/dimi/Downloads/SESA/test/"
for arquivo in os.listdir(diretorio):
    arrayDuracoes.append(librosa.get_duration(filename=diretorio + arquivo))

diretorio = "/home/dimi/Downloads/SESA/train/"
for arquivo in os.listdir(diretorio):
    arrayDuracoes.append(librosa.get_duration(filename=diretorio + arquivo))

In [ ]:
print("Duração mínima:", min(arrayDuracoes))
print("Duração máxima:", max(arrayDuracoes))

A célula abaixo vai setar o valor da variável do janelamento. FRAME_TIME e OVERLAP_TIME estarão em segundos, já FRAME_LENGTH e OVERLAP_LENGTH estarão em amostras.

In [2]:
audioTesteDir = "/home/dimi/Downloads/Datasets/SESA/SESA_Normalizado/test/casual_015.wav" 
audioTeste, freqAmostragem = librosa.load(audioTesteDir, sr=None, mono=True)

# frameTime     = min(arrayDuracoes)
# overlapTime   = frameTime / 2

# frameLength   = int(freqAmostragem * frameTime)
# overlapLength = int(freqAmostragem * overlapTime)

frameLength    = 17145
overlapLength  = 8572

print("Frequência de amostragem do dataset:\t", freqAmostragem)
print("Tamanho do janelamento (amostras):\t", frameLength)

Frequência de amostragem do dataset:	 16000
Tamanho do janelamento (amostras):	 17145


## Criando e testando as funções que vão extrair as features

Para testar as funções, vou utilizar um áudio qualquer, que já foi definido na célula anterior.

In [3]:
print("Tamanho do áudio teste (amostras):\t", len(audioTeste))
print("Tamanho do janelamento (amostras):\t", frameLength)

# PARA SABER O NÚMERO DE JANELAS, TENHO QUE MULTIPLICAR POR 2 ALÍ PQ AINDA TEM A SOBREPOSIÇÃO
# E SOMAR UM PQ SEMPRE VAI SOBRAR UM TECO
print("\nQtd de janelas do áudio de teste:\t", 1 + 2*int(len(audioTeste)/frameLength))

Tamanho do áudio teste (amostras):	 59443
Tamanho do janelamento (amostras):	 17145

Qtd de janelas do áudio de teste:	 7


### Features unitárias por janela

Primeiro, vou criar as funções para extrair as features que retornam somente um valor para cada janela do áudio. Vou deixar MFCCs, Chromas e Contrastes pra depois.

#### RMS

In [4]:
def extrairRMS(sinal, frameLength, overlapLength):
    return librosa.feature.rms(y=sinal, frame_length=frameLength, hop_length=overlapLength)

In [5]:
rms = extrairRMS(audioTeste, frameLength, overlapLength)
print("Total:", len(rms[0]))
print(rms)

Total: 7
[[1.8244174  1.7309765  1.2711647  0.64006954 0.4016891  0.21991439
  0.1802007 ]]


#### Centróide Espectral

In [6]:
def extrairCentroideEspectral(sinal, freqAmostragem, frameLength, overlapLength):
    return librosa.feature.spectral_centroid(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

In [7]:
centroideEspectral = extrairCentroideEspectral(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Total:", len(centroideEspectral[0]))
print(centroideEspectral)

Total: 7
[[1707.30728685 1970.68697732 1777.47750879 1628.37444789 1595.59947744
  1651.58124723 1589.94796039]]


#### Largura de banda espectral

In [8]:
def extrairLarguraBanda(sinal, freqAmostragem, frameLength, overlapLength):
    return librosa.feature.spectral_bandwidth(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

In [9]:
larguraBanda = extrairLarguraBanda(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Total:", len(larguraBanda[0]))
print(larguraBanda)

Total: 7
[[1769.53946856 1796.04157924 1652.75762193 1556.0451078  1638.72081291
  1831.41487789 2008.99462475]]


#### Planicidade espectral

In [10]:
def extrairPlanicidade(sinal, frameLength, overlapLength):
    return librosa.feature.spectral_flatness(y=sinal, n_fft=frameLength, hop_length=overlapLength)

In [11]:
planicidade = extrairPlanicidade(audioTeste, frameLength, overlapLength)
print("Total:", len(planicidade[0]))
print(planicidade)

Total: 7
[[0.02452501 0.08477717 0.05941166 0.04232679 0.04525724 0.04924234
  0.03320206]]


#### Rolloff espectral

In [12]:
def extrairRolloff(sinal, freqAmostragem, frameLength, overlapLength):
    return librosa.feature.spectral_rolloff(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

In [13]:
rolloff = extrairRolloff(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Total:", len(rolloff[0]))
print(rolloff)

Total: 7
[[3749.88334111 4061.59589361 3585.62762483 3187.12085861 3118.9920672
  3328.04479701 3695.75361643]]


#### Taxa de cruzamentos por zero

In [14]:
def extrairZCR(sinal, frameLength, overlapLength):
    return librosa.feature.zero_crossing_rate(y=sinal, frame_length=frameLength, hop_length=overlapLength)

In [15]:
zcr = extrairZCR(audioTeste, frameLength, overlapLength)
print("Total:", len(zcr[0]))
print(zcr)

Total: 7
[[0.06310878 0.14237387 0.15263925 0.13957422 0.12015165 0.09932925
  0.07588218]]


### Features não unitárias por janela

Agora sim é hora de extrair os MFCCs, Chromas e Contrastes.

#### MFCC

A função do MFCC retorna uma matriz, e ela será necessária para calcular os deltas e delta deltas, mas, na verdade, o MFCC que será colocado no array de features é uma média de cada linha da matriz. Por isso, primeiro vamos criar uma matriz do MFCC, e só depois a função extrairMediaMFCC vai fazer a média.

In [16]:
def extrairMatrizMFCC(sinal, freqAmostragem):
    return librosa.feature.mfcc(y=sinal, sr=freqAmostragem)

In [17]:
matrizMFCC = extrairMatrizMFCC(audioTeste, freqAmostragem)

print("Qtd linhas (MFCCs):", len(matrizMFCC))
print("Qtd de valores em cada linha:", len(matrizMFCC[0]))

Qtd linhas (MFCCs): 20
Qtd de valores em cada linha: 117


In [18]:
def extrairMFCCs(matrizMFCC):
    
    arrayMFCCs = []
    
    for linha in matrizMFCC:
        arrayMFCCs.append(np.mean(linha))
        
    return arrayMFCCs

In [19]:
MFCCs = extrairMFCCs(matrizMFCC)

print("Qtd de MFCCs:", len(MFCCs))
print(MFCCs)

Qtd de MFCCs: 20
[90.35726, 96.21592, -12.690334, 23.92078, 3.9557667, 9.098954, -1.1837934, 1.7929257, -3.8612175, 2.1821105, -1.5066134, 1.5684648, -2.1820452, -0.8958572, -5.939487, -2.4142895, -2.9936216, -0.32899868, -1.2340535, -0.96812487]


#### MFCC Delta

Assim como o MFCC, a função do librosa que extrai os deltas retorna uma matriz. A função abaixo retorna a média de cada linha dessa matriz.

In [20]:
def extrairDeltas(matrizMFCC):
    matrizDelta = librosa.feature.delta(matrizMFCC, order=1)

    arrayDelta = []

    for linha in matrizDelta:
        arrayDelta.append(np.mean(linha))

    return arrayDelta

In [21]:
deltas = extrairDeltas(matrizMFCC)

print("Qtd de deltas:", len(deltas))
print(deltas)

Qtd de deltas: 20
[1.2634532, 0.2583152, -0.18495567, -0.121523894, -0.084837, -0.056756202, -0.15059838, -0.0023076106, 0.008575201, 0.014145123, -0.09964522, -0.036289196, -0.025149047, -0.005005702, -0.01031554, -0.0028545063, -0.020311967, 0.08247934, 0.026181314, 0.00038621976]


#### MFCC Delta Delta

In [22]:
def extrairDeltaDeltas(matrizMFCC):
    matrizDeltaDelta = librosa.feature.delta(matrizMFCC, order=2)

    arrayDeltaDelta = []

    for linha in matrizDeltaDelta:
        arrayDeltaDelta.append(np.mean(linha))

    return arrayDeltaDelta

In [23]:
deltaDeltas = extrairDeltaDeltas(matrizMFCC)

print("Qtd de delta deltas:", len(deltaDeltas))
print(deltaDeltas)

Qtd de delta deltas: 20
[0.4892378, -0.021952502, -0.043641336, 0.044424415, 0.0073836413, 0.045231894, -0.0027372497, -0.009363727, -0.027617775, 0.032582693, 0.008763685, -0.009184939, -0.02254185, 0.0071497113, -0.008626881, 0.030296305, -0.00078415044, -0.032558195, -0.054833964, -0.0201306]


#### Mel Espectrograma

Essa também retorna uma matriz, mas ela vem invertida, tenho que tirar a média de cada COLUNA.

In [24]:
def extrairMelEspectrograma(sinal, freqAmostragem, frameLength, overlapLength):
    
    matrizMelEspectrograma = librosa.feature.melspectrogram(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
    
    arrayMelEspectrograma = []
    
    for coluna in matrizMelEspectrograma.T:
        arrayMelEspectrograma.append(np.mean(coluna))
    
    return arrayMelEspectrograma

In [25]:
melEspectrograma = extrairMelEspectrograma(audioTeste, freqAmostragem, frameLength, overlapLength)

print("Qtd de features:", len(melEspectrograma))
print(melEspectrograma)

Qtd de features: 7
[19364.988, 72401.11, 14928.781, 7237.6953, 1758.3064, 666.27026, 586.1963]


#### Cromagrama

Essa também retorna uma matriz. Ela tem 12 cromagramas (linhas). Vou tirar a média de cada linha.

In [26]:
def extrairCromagramas(sinal, freqAmostragem, frameLength, overlapLength):
    
    matrizCromagramas = librosa.feature.chroma_stft(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
    
    arrayCromagramas = []
    
    for linha in matrizCromagramas:
        arrayCromagramas.append(np.mean(linha))
    
    return arrayCromagramas

In [27]:
cromagramas = extrairCromagramas(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Features:", len(cromagramas))
print(cromagramas)

Features: 12
[0.7875995, 0.789555, 0.75121117, 0.69452906, 0.7602438, 0.8831463, 0.82073545, 0.8278985, 0.76663816, 0.73204404, 0.79973537, 0.7402423]


#### Cromagrama de constante Q

Essa função não precisa do tamanho da janela (frameLength) e a sobreposição: "hop_length must be a positive integer multiple of 2^6 for 7-octave CQT". Portanto, vou deixar esse parâmetro como padrão.

In [28]:
def extrairCromagramasQ(sinal, freqAmostragem):
    
    matrizCromagramasQ = librosa.feature.chroma_cqt(y=sinal, sr=freqAmostragem)
    
    arrayCromagramasQ = []
    
    for linha in matrizCromagramasQ:
        arrayCromagramasQ.append(np.mean(linha))
    
    return arrayCromagramasQ

In [29]:
cromagramasQ = extrairCromagramasQ(audioTeste, freqAmostragem)
print("Features:", len(cromagramasQ))
print(cromagramasQ)

Features: 12
[0.7928085000434514, 0.7942605753931457, 0.7598130786874815, 0.738256730346462, 0.7696198297598498, 0.7740035970465032, 0.7500368186214228, 0.7563676292237278, 0.6885176645831349, 0.632005522003134, 0.6410327953833604, 0.5981624320001039]


#### Croma CENS

Assim como o cromagrama de constante Q, essa função não precisa do tamanho da janela (frameLength) e a sobreposição: "hop_length must be a positive integer multiple of 2^6 for 7-octave CQT". Portanto, vou deixar esse parâmetro como padrão.

In [30]:
def extrairCromaCENSs(sinal, freqAmostragem):
    
    matrizCromaCENSs = librosa.feature.chroma_cens(y=sinal, sr=freqAmostragem)
    
    arrayCromaCENSs = []
    
    for linha in matrizCromaCENSs:
        arrayCromaCENSs.append(np.mean(linha))
    
    return arrayCromaCENSs

In [31]:
cromaCENSs = extrairCromaCENSs(audioTeste, freqAmostragem)
print("Features:", len(cromaCENSs))
print(cromaCENSs)

Features: 12
[0.33069522167624155, 0.32728316903120086, 0.29582167844150786, 0.29450791898182055, 0.3111067782230877, 0.3072401050430688, 0.296477005846955, 0.2942989525684849, 0.2671905424505684, 0.23553858905577774, 0.24617813448804013, 0.21002142719909053]


#### Contraste espectral

As ultimas features serão essas.

In [32]:
def extrairContrastes(sinal, freqAmostragem, frameLength, overlapLength):
    
    matrizContrastes = librosa.feature.spectral_contrast(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
    
    arrayConstrastes = []
    
    for linha in matrizContrastes:
        arrayConstrastes.append(np.mean(linha))
    
    return arrayConstrastes

In [33]:
contrastes = extrairContrastes(audioTeste, freqAmostragem, frameLength, overlapLength)
print(contrastes)

[18.187328876949742, 15.745184288522415, 15.827735450971762, 15.369437040530203, 16.12093632251706, 17.10694956632393, 16.748398695995743]


#### Tonnetz

In [36]:
def extrairTonnetz(sinal, freqAmostragem):
    
    matrizTonnetz = librosa.feature.tonnetz(y=sinal, sr=freqAmostragem)

    arrayTonnetz = []

    for linha in matrizTonnetz:
        arrayTonnetz.append(np.mean(linha))

    return arrayTonnetz

In [37]:
extrairTonnetz(audioTeste, freqAmostragem)

[-0.004656962928969554,
 0.015397312645967883,
 -0.011890796281157972,
 0.0119155427492697,
 0.007296877893321908,
 0.0014926964149470884]

## Criando a classe

O objetivo é que essa classe entre numa pasta e me devolva uma matriz em que cada linha seja um áudio e cada coluna seja uma feature. Essa classe ainda deve ser capaz de gravar tudo isso num csv.

As features serão extraídas, e depois que tudo já estiver pronto, ela vai escalonar as features e depois passar tudo num PCA.

Então, para usar a classe, é necessário ter as pastas de áudios muito bem organizadas para que ela crie o CSV dentro da pasta que se queira. Preferi fazer assim, pois, se eu fosse abrir todos os áudios numa matriz do python pra só depois mandar pra essa classe, poderia ficar mto pesado. É melhor que ela só receba o caminho pra uma pasta e gere um CSV.

### Criando funções intermediárias

Vou usar essa parte pra criar as funções intermediárias, que não serão responsáveis por calcular nenhuma feature, mas apenas montar os arrays e metrizes da classe.

#### Função que faz o janelamento

Existe um problema em deixar que as funções de extração de features criadas acima façam o janelamento: ao invés de retornarem valores unitários para as features, elas vão retornar um array em que cada posição representa um janelamento. Portanto, a solução é fazer o janelamento antes de extrair as features e deixar para mandar para essas funções apenas as janelas, fazendo com que frameLength seja igual ao tamanho da janela que está sendo enviada e que overlapLength seja 0.

A função abaixo usa a função frame do librosa que retorna as janelas como COLUNAS. Como eu quero que cada janela seja uma LINHA, eu retorno a transposta dessa função.

In [ ]:
def fazerJanelamento(sinal, frameLength, overlapLength):
    return librosa.util.frame(sinal, frame_length=frameLength, hop_length=overlapLength).T

In [ ]:
framesAudioTeste = fazerJanelamento(audioTeste, frameLength, overlapLength)
print("Qtd janelas:", len(framesAudioTeste))
print(framesAudioTeste)
print(framesAudioTeste[0])

In [ ]:
Audio(data=framesAudioTeste[0], rate=freqAmostragem)

#### Função que extrai features de um único frame

Em algum momento, a classe deverá passar em cada um dos áudios da pasta para ir extraindo as features. A próxima função extrai as features de um único frame de áudio e retorna um array com essas features. Posteriormente esse array deverá ser integrado à matriz de features de todos os áudios.

Haverá uma outra função para extrair as features de um único áudio. Ela deverá pegar um áudio, usar a função de janelamento, e usar a função abaixo para extrair as features de cada uma das janelas. Depois, ela vai retornar uma matriz com as features de cada frame de um único áudio.

PARA IMPEDIR QUE O LIBROSA CONTINUE FAZENDO O JANELAMENTO DO ÁUDIO MESMO QUE frameLength SEJA DO TAMANHO DO ÁUDIO, O PARÂMETRO DE OVERLAP DEVE SER MAIOR OU IGUAL A frameLength.

In [ ]:
def extrairFeaturesUnicoFrame(sinal, freqAmostragem, frameLength):
    
    # PARA IMPEDIR QUE O LIBROSA CONTINUE FAZENDO O JANELAMENTO DO ÁUDIO MESMO QUE frameLength 
    # SEJA DO TAMANHO DO ÁUDIO, O PARÂMETRO DE OVERLAP DEVE SER MAIOR OU IGUAL A frameLength:
    overlapLength = frameLength
    
    # CRIANDO O ARRAY DE FEATURES DO FRAME EM QUESTAO
    arrayFeaturesFrame = []
    
    #PRIMEIRO, VOU EXTRAIR AS FEATURES UNITARIAS
    arrayFeaturesFrame.append(float(extrairRMS(sinal, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairCentroideEspectral(sinal, freqAmostragem, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairLarguraBanda(sinal, freqAmostragem, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairPlanicidade(sinal, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairRolloff(sinal, freqAmostragem, frameLength, overlapLength)))
    arrayFeaturesFrame.append(float(extrairZCR(sinal, frameLength, overlapLength)))
    
    # AGORA VAMOS PASSAR PARA AS NAO UNITARIAS, PRIMEIRO, E PRECISO CRIAR A MATRIZ DOS MFCCS
    matrizMFCC          = extrairMatrizMFCC(sinal, freqAmostragem)
    
    # AGORA SIM EU SAIO EXTRAINDO AS FEATURES 
    arrayFeaturesFrame += extrairMFCCs(matrizMFCC)
    arrayFeaturesFrame += extrairDeltas(matrizMFCC)
    arrayFeaturesFrame += extrairDeltaDeltas(matrizMFCC)
    arrayFeaturesFrame += extrairMelEspectrograma(sinal, freqAmostragem, frameLength, overlapLength)
    arrayFeaturesFrame += extrairCromagramas(sinal, freqAmostragem, frameLength, overlapLength)
    arrayFeaturesFrame += extrairCromagramasQ(sinal, freqAmostragem)
    arrayFeaturesFrame += extrairCromaCENSs(sinal, freqAmostragem)
    arrayFeaturesFrame += extrairContrastes(sinal, freqAmostragem, frameLength, overlapLength)
    
    # POR FIM, RETORNO O ARRAY DE FEATURES DO AUDIO QUE FOI ENVIADO PARA ESSA FUNCAO
    return arrayFeaturesFrame

In [ ]:
frameAtual = framesAudioTeste[0]

featuresFrameAtual = extrairFeaturesUnicoFrame(frameAtual, freqAmostragem, frameLength)

print(len(featuresFrameAtual))
print(featuresFrameAtual)

#### Função que extrai as features de um único áudio

Essa função vai pegar um áudio, usar a função de janelamento, e para cada janela do áudio em questão, ela vai usar a função de extrair as features de uma única janela (implementada acima). Depois, ela vai retornar uma matriz com as features de cada frame de um único áudio, onde cada linha é uma frame e cada coluna é uma feature.

In [ ]:
def extrairFeaturesUnicoAudio(sinal, freqAmostragem, frameLength, overlapLength):
    
    # PRIMEIRO, VOU CRIAR A MATRIZ QUE VAI CONTER AS FEATURES DE CADA JANELA
    # CADA LINHA E UMA JANELA E CADA COLUNA E UMA FEATURE
    matrizFeaturesAudio = []
    
    # DEPOIS, VOU FAZER O JANELAMENTO
    matrizFramesAudio = fazerJanelamento(sinal, frameLength, overlapLength)
    
    # AGORA, PARA CADA JANELA, VOU EXTRAIR AS FEATURES E COLOCAR COMO UMA LINHA NOVA NA MATRIZ
    for frameAtual in matrizFramesAudio:
        matrizFeaturesAudio.append(extrairFeaturesUnicoFrame(frameAtual, freqAmostragem, frameLength))
    
    # RETORNO A MATRIZ DE FEATURES DESSE AUDIO
    return matrizFeaturesAudio

In [ ]:
matrizFeaturesAudioTeste = extrairFeaturesUnicoAudio(audioTeste, freqAmostragem, frameLength, overlapLength)
print("Qtd frames (linhas):", len(matrizFeaturesAudioTeste))
print("Qtd features (colunas):", len(matrizFeaturesAudioTeste[0]))

#### Função que coloca o nome do arquivo e a classificação correta na matriz de features de um áudio

Haverá uma matriz de dados que terá as seguintes colunas: **nomeArquivo, ...features... e classificacaoCorreta**. Mas, a função implementada para gerar a matriz de features de um único áudio **extrairFeaturesUnicoAudio** devolve uma matriz de features **sem** o nome do áudio e a classificação correta. Portanto, a função abaixo apenas pega essa última matriz e coloca o nome do arquivo no começo e a classificação correta ao final. Posteriormente, o resultado será agregado à matriz de dados citada em primeiro lugar.

ESSA FUNÇÃO RETORNA UM DATAFRAME PANDAS, NÃO UMA MATRIZ QUALQUER.

https://www.geeksforgeeks.org/python-pandas-dataframe-insert/

In [ ]:
def adicionarNomeArquivoEClasse(matrizFeaturesAudioAtual, nomeArquivo, classificacaoCorreta):
    # PRIMEIRO TRANSFORMO A MATRIZ NUM PANDAS DATAFRAME
    dataframeAudioAtual = pd.DataFrame(matrizFeaturesAudioAtual)
    
    # AGORA COLOCO A COLUNA DO NOME NO COMECO (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
    dataframeAudioAtual.insert(0, "nomeArquivo", nomeArquivo, True)
    
    # AGORA COLOCO A COLUNA DA CLASSIFICACAO CORRETA NA ULTIMA POSICAO
    dataframeAudioAtual.insert(len(dataframeAudioAtual.columns), "classificacaoCorreta", classificacaoCorreta, True)
    
    return dataframeAudioAtual

In [ ]:
dataframeAudioTeste = adicionarNomeArquivoEClasse(matrizFeaturesAudioTeste, "teste.wav", "gunshot")
dataframeAudioTeste

#### Função para verificar qual é a classificação correta de acordo com o nome do arquivo

Essa é bem básica. Em algum determinado momento eu vou precisar saber a classficação correta de um determinado áudio. Eu só consigo saber isso pelo nome do arquivo. LEMBRANDO QUE TODO O CÓDIGO ESCRITO AQUI SERVE PARA O BANCO DE DADOS SESA. Os arquivos são nomeados como **classe_contador.wav**. Por exemplo: casual_000.wav, explosion_032.wav e gunshot_032.wav.

In [ ]:
def verificarClassificacaoCorreta(nomeArquivo):
    arrayNome = nomeArquivo.split("_")
    return arrayNome[0]

In [ ]:
diretorio = "/home/dimi/Downloads/SESA/test/"
parar = 10
for nomeArquivo in os.listdir(diretorio):
    print(verificarClassificacaoCorreta(nomeArquivo))
    parar -= 1
    if parar == 0:
        break

#### Função que passa por todos os áudios da pasta e vai montando o dataframe

Essa é função que une todas as outras. Ela vai passar por todos os áudios da pasta, verificar qual é a classificação correta de cada áudio, extrair as features, gerar o dataframe de features do áudio atual e agregar ao dataframe de todos os áudios.

In [ ]:
def montarDataframeTodosOsAudios(diretorio, freqAmostragem, frameLength, overlapLength):
    
    print("Fora da classe")
    
    # CRIANDO O ARRAY COM O NOME DOS ARQUIVOS
    arrayNomeArquivos = os.listdir(diretorio)
    
    # PEGANDO O TOTAL DE ARQUIVOS NA PASTA APENAS PARA FINS DE PRINT
    totalArquivosNaPasta = len(arrayNomeArquivos)
    
    # ABAIXO, VOU CRIAR O DATAFRAME DE TODOS OS AUDIOS
    dataframeGeral = pd.DataFrame()

    # VOU PASSAR POR TODOS OS AUDIOS DO DIRETORIO
    for i, nomeArquivo in enumerate(arrayNomeArquivos):
        # PRINTANDO O PROGRESSO
        print("Extraindo features do arquivo", i+1, "de", totalArquivosNaPasta, "-> " + str(100*((i+1)/totalArquivosNaPasta)) + "%")
        
        # ABRO O AUDIO ATUAL COM O LIBROSA
        audioAtual, freqAmostragem = librosa.load(audioTesteDir, sr=freqAmostragem, mono=True) 
        
        # VERIFICAO QUAL E A CLASSIFICACAO CORRETA
        classeAudioAtual = verificarClassificacaoCorreta(nomeArquivo)
        
        # MONTO A MATRIZ DE FEATUREA DE CADA FRAME DO AUDIO ATUAL (a funcao abaixo
        # devolve uma matriz normal e faz o janelamento em outra funcao tb)
        matrizFeaturesAudioAtual = extrairFeaturesUnicoAudio(audioAtual, freqAmostragem, frameLength, overlapLength)
        
        # AGORA E HORA DE COLOCAR O NOME E A CLASSIFICACAO CORRETA NA MATRIZ
        # MAAAS, NA FUNCAO ABAIXO, O BICHO VIRA UM DATAFRAME PANDAS
        dataframeAudioAtual      = adicionarNomeArquivoEClasse(matrizFeaturesAudioAtual, nomeArquivo, classeAudioAtual)
        
        # CONCATENANDO O DATAFRAME DO AUDIO ATUAL AO DATAFRAME GERAL
        dataframeGeral = pd.concat([dataframeGeral, dataframeAudioAtual])
        
    # POR ULTIMO, RETORNO O DATAFRAME GERAL
    return dataframeGeral

#### Função para escalonar as features

Essa função recebe o dataframe geral, remove as primeira e última colunas (nome e classificação), faz o escalonamento e depois coloca as colunas de nome e classificação de volta.

In [ ]:
def escalonarFeatures(dataframeGeral):
    
    print("Escalonando features")
    
    # COPIANDO AS COLUNAS DE NOME E CLASSIFICACAO
    colunaArquivo       = dataframeGeral["nomeArquivo"]
    colunaClassificacao = dataframeGeral["classificacaoCorreta"]
    
    # DELETANDO AS COLUNAS ARQUIVO E CLASSIFICACAO
    dataframeGeral = dataframeGeral.drop(['nomeArquivo', 'classificacaoCorreta'], axis=1)
    
    # ESCALONANDO
    dataframeGeral = pd.DataFrame(StandardScaler().fit_transform(dataframeGeral))
    
    # COM ESSA COISA DE TIRA E POE COLUNA, O PANDAS NAO SABE LINDAR COM OS INDEXES,
    # ABAIXO EU ESTOU RESETANDO TUDO
    colunaArquivo.reset_index(inplace=True, drop=True)
    colunaClassificacao.reset_index(inplace=True, drop=True)
    dataframeGeral.reset_index(inplace=True, drop=True)
    
    # ADICIONANDO AS COLUNAS QUE FORAM EXCLUIDAS (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
    dataframeGeral.insert(0, "nomeArquivo", colunaArquivo, True)
    dataframeGeral.insert(len(dataframeGeral.columns), "classificacaoCorreta", colunaClassificacao, True)
    
    return dataframeGeral

#### Função que reduz a dimensionalidade do dataframe geral

Depois de escalonar, vamos reduzir a dimensionalidade utilizando PCA. A ideia é fazer com que o CSV já saia pronto para ser usado, logo, a redução de dimensionalidade deve vir nessa classe. Apesar disso, vou deixar o parâmetro **nDimensoes** lá no construtor da classe, ai caso ele seja nulo eu pulo a parte da redução de dimensionalidade e gero o CSV com todas as features.

Poder usar a classe sem fazer a redução de dimensionalidade vai ser muito importante para verificar qual é o melhor número de componentes principais do PCA, como se mostra https://towardsdatascience.com/an-approach-to-choosing-the-number-of-components-in-a-principal-component-analysis-pca-3b9f3d6e73fe

Então a ideia é primeiro gerar o CSV completo dos dados de treino, sem reduzir a dimensionalidade, pra depois fazer um estudo sobre o melhor número de componentes principais. Aí sim, com **nDimensoes** definido, eu começo a usar a classe com essa função.

In [ ]:
def reduzirDimensionalidade(dataframeGeral, nDimensoes):
    
    print("Reduzindo a dimensionalidade")
    
    # COPIANDO AS COLUNAS DE NOME E CLASSIFICACAO
    colunaArquivo       = dataframeGeral["nomeArquivo"]
    colunaClassificacao = dataframeGeral["classificacaoCorreta"]
    
    # DELETANDO AS COLUNAS ARQUIVO E CLASSIFICACAO
    dataframeGeral = dataframeGeral.drop(['nomeArquivo', 'classificacaoCorreta'], axis=1)
    
    # REDUZINDO A DIMENSIONALIDADE
    dataframeGeral = pd.DataFrame(PCA(n_components=nDimensoes).fit_transform(dataframeGeral))
    
    # COM ESSA COISA DE TIRA E POE COLUNA, O PANDAS NAO SABE LINDAR COM OS INDEXES,
    # ABAIXO EU ESTOU RESETANDO TUDO
    colunaArquivo.reset_index(inplace=True, drop=True)
    colunaClassificacao.reset_index(inplace=True, drop=True)
    dataframeGeral.reset_index(inplace=True, drop=True)
    
    # ADICIONANDO AS COLUNAS QUE FORAM EXCLUIDAS (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
    dataframeGeral.insert(0, "nomeArquivo", colunaArquivo, True)
    dataframeGeral.insert(len(dataframeGeral.columns), "classificacaoCorreta", colunaClassificacao, True)
    
    return dataframeGeral

### Finalizando

Abaixo, vou criar a função que será a construtora da classe (init). Ela vai usar todas as funções que já foram criadas. Depois disso, vai bastar copiar todas as funções pra dentro da classe e fazer acontecer.

In [ ]:
def construtor(diretorio, freqAmostragem, frameLength, overlapLength, nDimensoes=None):
    
    # PRIMEIRO EU CRIO O DATAFRAME DE TODOS OS AUDIOS DA PASTA
    dataframeGeral = montarDataframeTodosOsAudios(diretorio, freqAmostragem, frameLength, overlapLength)
    
    # DEPOIS EU ESCALONO AS FEATURES
    dataframeGeral = escalonarFeatures(dataframeGeral)
    
    # E AGORA EU REDUZO A DIMENSIONALIDADE
    if nDimensoes != None:
        dataframeGeral = reduzirDimensionalidade(dataframeGeral, nDimensoes)
        
    # PARA FINALIZAR, VOU ESCREVER O DATAFRAME NUM CSV
    nomeCSV = diretorio + str(time.time()) + ".csv"
    print("Escrevendo CSV:", nomeCSV)
    dataframeGeral.to_csv(nomeCSV, index=False)
    
    # TAMBEM VOU RETORNAR O DATAFRAME, PQ VAI QUE EU PRECISO NE
    print("Operação finalizada")
    return dataframeGeral

# FINALMENTE COLOCANDO TUDO DENTRO DA CLASSE

A primeira parte vai ter o construtor da classe. A segunda parte vai ser das funções intermediárias que fazem tudo acontecer, e só por último vou colocar as funções de extração de features.

In [ ]:
# TODA A IMPLEMETACAO DESSA CLASSE ESTA MUITO BEM DOCUMENTADA NO JUPYTER "
# Implementação de classe para extração de features - SESA Dataset"
# VOU COLOCAR O MAXIMO DE COMENTARIOS POSSIVEIS AQUI TAMBEM, MAS PARA MAIS INFORMACOES
# E MELHOR OLHAR POR LA

# COMO USAR A CLASSE:
# A CLASSE ENTRA EM UMA DETERMINADA PASTA CONTENDO APENAS ARQUIVOS WAV E CRIA UM CSV DE FEATURES
# A CLASSE TAMBEM E RESPONSAVEL POR ESCALONAR AS FEATURES E FAZER UMA REDUCAO DE DIMENSIONALIDADE

# PRIMEIRO, OS PARAMETROS ABAIXOS DEVEM SER SETADOS:
# diretorio      -> PASTA ONDE A CLASSE VAI PROCURAR PELOS WAVs PARA GERAR O CSV
# freqAmostragem -> A FREQUENCIA DE AMOSTRAGEM DOS AUDIOS DESSA PASTA
# frameLength    -> O TAMANHO DAS JANELAS DE CADA AUDIO EM QTD DE FRAMES
# overlapLength  -> TAMANHO DA SOBREPOSICAO EM QTD DE FRAMES
# nDimensoes     -> QTD DE DIMENSOES UTILIZADAS NO PCA. CASO SEJA NULL, NAO HAVERA REDUCAO DE DIMENSIONALIDADE

# O COMANDO ABAIXO INSTANCIA A CLASSE. O CONSTRUTOR DEVOLVE UM DATAFRAME PANDAS AO MESMO TEMPO QUE CRIA O CSV
# dataframeGeral = extrairFeatures(diretorio, freqAmostragem, frameLength, overlapLength, nDimensoes)

import os
import librosa
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

class ExtrairFeatures:

	# CONSTRUTOR---------------------------------------------------------------------------------------------
	def __init__(self, diretorio, freqAmostragem, frameLength, overlapLength, escalonamento=True, nDimensoes=None):    
		# PRIMEIRO EU CRIO O DATAFRAME DE TODOS OS AUDIOS DA PASTA
		dataframeGeral = self.montarDataframeTodosOsAudios(diretorio, freqAmostragem, frameLength, overlapLength)
		
		# DEPOIS EU ESCALONO AS FEATURES
		if escalonamento == True:
			dataframeGeral = self.escalonarFeatures(dataframeGeral)
		
		# E AGORA EU REDUZO A DIMENSIONALIDADE
		if nDimensoes != None:
			dataframeGeral = self.reduzirDimensionalidade(dataframeGeral, nDimensoes)
			
		# PARA FINALIZAR, VOU ESCREVER O DATAFRAME NUM CSV
		nomeCSV = diretorio + str(time.time()) + ".csv"
		print("Escrevendo CSV:", nomeCSV)
		dataframeGeral.to_csv(nomeCSV, index=False)
		
		# TAMBEM VOU RETORNAR O DATAFRAME, PQ VAI QUE EU PRECISO NE
		print("Operação finalizada")
		
		# return dataframeGeral

	# DEFINICAO DE FUNCOES INTERMEDIARIAS--------------------------------------------------------------------	
	def fazerJanelamento(self, sinal, frameLength, overlapLength):
		# Função que faz o janelamento
		# Existe um problema em deixar que as funções de extração de features criadas acima façam o janelamento: 
		# ao invés de retornarem valores unitários para as features, elas vão retornar um array em que cada posição 
		# representa um janelamento. Portanto, a solução é fazer o janelamento antes de extrair as features e deixar 
		# para mandar para essas funções apenas as janelas, fazendo com que frameLength seja igual ao tamanho da 
		# janela que está sendo enviada e que overlapLength seja 0.
		# A função abaixo usa a função frame do librosa que retorna as janelas como COLUNAS. 
		# Como eu quero que cada janela seja uma LINHA, eu retorno a transposta dessa função.
		return librosa.util.frame(sinal, frame_length=frameLength, hop_length=overlapLength).T

	def extrairFeaturesUnicoFrame(self, sinal, freqAmostragem, frameLength):
		# Função que extrai features de um único frame
		# Em algum momento, a classe deverá passar em cada um dos áudios da pasta para ir extraindo as features. 
		# A próxima função extrai as features de um único frame de áudio e retorna um array com essas features. 
		# Posteriormente esse array deverá ser integrado à matriz de features de todos os áudios.
		# Haverá uma outra função para extrair as features de um único áudio. Ela deverá pegar um áudio, usar a 
		# função de janelamento, e usar a função abaixo para extrair as features de cada uma das janelas. Depois, 
		# ela vai retornar uma matriz com as features de cada frame de um único áudio.
		# 
		# PARA IMPEDIR QUE O LIBROSA CONTINUE FAZENDO O JANELAMENTO DO ÁUDIO MESMO QUE frameLength 
		# SEJA DO TAMANHO DO ÁUDIO, O PARÂMETRO DE OVERLAP DEVE SER MAIOR OU IGUAL A frameLength.s
		overlapLength = frameLength
		
		# CRIANDO O ARRAY DE FEATURES DO FRAME EM QUESTAO
		arrayFeaturesFrame = []
		
		#PRIMEIRO, VOU EXTRAIR AS FEATURES UNITARIAS
		arrayFeaturesFrame.append(float(self.extrairRMS(sinal, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairCentroideEspectral(sinal, freqAmostragem, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairLarguraBanda(sinal, freqAmostragem, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairPlanicidade(sinal, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairRolloff(sinal, freqAmostragem, frameLength, overlapLength)))
		arrayFeaturesFrame.append(float(self.extrairZCR(sinal, frameLength, overlapLength)))
		
		# AGORA VAMOS PASSAR PARA AS NAO UNITARIAS, PRIMEIRO, E PRECISO CRIAR A MATRIZ DOS MFCCS
		matrizMFCC          = self.extrairMatrizMFCC(sinal, freqAmostragem)
		
		# AGORA SIM EU SAIO EXTRAINDO AS FEATURES 
		arrayFeaturesFrame += self.extrairMFCCs(matrizMFCC)
		arrayFeaturesFrame += self.extrairDeltas(matrizMFCC)
		arrayFeaturesFrame += self.extrairDeltaDeltas(matrizMFCC)
		arrayFeaturesFrame += self.extrairMelEspectrograma(sinal, freqAmostragem, frameLength, overlapLength)
		arrayFeaturesFrame += self.extrairCromagramas(sinal, freqAmostragem, frameLength, overlapLength)
		arrayFeaturesFrame += self.extrairCromagramasQ(sinal, freqAmostragem)
		arrayFeaturesFrame += self.extrairCromaCENSs(sinal, freqAmostragem)
		arrayFeaturesFrame += self.extrairContrastes(sinal, freqAmostragem, frameLength, overlapLength)
		
		# POR FIM, RETORNO O ARRAY DE FEATURES DO AUDIO QUE FOI ENVIADO PARA ESSA FUNCAO
		return arrayFeaturesFrame

	def extrairFeaturesUnicoAudio(self, sinal, freqAmostragem, frameLength, overlapLength):
		# Função que extrai as features de um único áudio
		# Essa função vai pegar um áudio, usar a função de janelamento, e para cada janela do áudio em questão, 
		# ela vai usar a função de extrair as features de uma única janela (implementada acima). 
		# Depois, ela vai retornar uma matriz com as features de cada frame de um único áudio, onde cada linha 
		# é uma frame e cada coluna é uma feature.

		# PRIMEIRO, VOU CRIAR A MATRIZ QUE VAI CONTER AS FEATURES DE CADA JANELA
		# CADA LINHA E UMA JANELA E CADA COLUNA E UMA FEATURE
		matrizFeaturesAudio = []
		
		# DEPOIS, VOU FAZER O JANELAMENTO
		matrizFramesAudio = self.fazerJanelamento(sinal, frameLength, overlapLength)
		
		# AGORA, PARA CADA JANELA, VOU EXTRAIR AS FEATURES E COLOCAR COMO UMA LINHA NOVA NA MATRIZ
		for frameAtual in matrizFramesAudio:
			matrizFeaturesAudio.append(self.extrairFeaturesUnicoFrame(frameAtual, freqAmostragem, frameLength))
		
		# RETORNO A MATRIZ DE FEATURES DESSE AUDIO
		return matrizFeaturesAudio

	def adicionarNomeArquivoEClasse(self, matrizFeaturesAudioAtual, nomeArquivo, classificacaoCorreta):
		#Função que coloca o nome do arquivo e a classificação correta na matriz de features de um áudio
		#Haverá uma matriz de dados que terá as seguintes colunas: nomeArquivo, ...features... e classificacaoCorreta. 
		#Mas, a função implementada para gerar a matriz de features de um único áudio extrairFeaturesUnicoAudio 
		#devolve uma matriz de features sem o nome do áudio e a classificação correta. Portanto, a função abaixo 
		# apenas pega essa última matriz e coloca o nome do arquivo no começo e a classificação correta ao final. 
		# Posteriormente, o resultado será agregado à matriz de dados citada em primeiro lugar.

		#ESSA FUNÇÃO RETORNA UM DATAFRAME PANDAS, NÃO UMA MATRIZ QUALQUER.

		#https://www.geeksforgeeks.org/python-pandas-dataframe-insert/

		# PRIMEIRO TRANSFORMO A MATRIZ NUM PANDAS DATAFRAME
		dataframeAudioAtual = pd.DataFrame(matrizFeaturesAudioAtual)
		
		# AGORA COLOCO A COLUNA DO NOME NO COMECO (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
		dataframeAudioAtual.insert(0, "nomeArquivo", nomeArquivo, True)
		
		# AGORA COLOCO A COLUNA DA CLASSIFICACAO CORRETA NA ULTIMA POSICAO
		dataframeAudioAtual.insert(len(dataframeAudioAtual.columns), "classificacaoCorreta", classificacaoCorreta, True)
		
		return dataframeAudioAtual

	def verificarClassificacaoCorreta(self, nomeArquivo):
		#Função para verificar qual é a classificação correta de acordo com o nome do arquivo
		#Essa é bem básica. Em algum determinado momento eu vou precisar saber a classficação 
		# correta de um determinado áudio. Eu só consigo saber isso pelo nome do arquivo. LEMBRANDO QUE 
		# TODO O CÓDIGO ESCRITO AQUI SERVE PARA O BANCO DE DADOS SESA. Os arquivos são nomeados 
		# como classe_contador.wav. Por exemplo: casual_000.wav, explosion_032.wav e gunshot_032.wav.
		arrayNome = nomeArquivo.split("_")
		return arrayNome[0]

	def montarDataframeTodosOsAudios(self, diretorio, freqAmostragem, frameLength, overlapLength):
		# Função que passa por todos os áudios da pasta e vai montando o dataframe
		# Essa é função que une todas as outras. Ela vai passar por todos os áudios da pasta, 
		# verificar qual é a classificação correta de cada áudio, extrair as features, gerar o 
		# dataframe de features do áudio atual e agregar ao dataframe de todos os áudios.
	
		# CRIANDO O ARRAY COM O NOME DOS ARQUIVOS
		arrayNomeArquivos = os.listdir(diretorio)
		
		# PEGANDO O TOTAL DE ARQUIVOS NA PASTA APENAS PARA FINS DE PRINT
		totalArquivosNaPasta = len(arrayNomeArquivos)
		
		# ABAIXO, VOU CRIAR O DATAFRAME DE TODOS OS AUDIOS
		dataframeGeral = pd.DataFrame()

		# VOU PASSAR POR TODOS OS AUDIOS DO DIRETORIO
		for i, nomeArquivo in enumerate(arrayNomeArquivos):
			# PRINTANDO O PROGRESSO
			print("Extraindo features do arquivo", i+1, "de", totalArquivosNaPasta, "-> " + str(100*((i+1)/totalArquivosNaPasta)) + "%")
			
			# ABRO O AUDIO ATUAL COM O LIBROSA
			audioAtual, freqAmostragem = librosa.load(diretorio+nomeArquivo, sr=freqAmostragem, mono=True) 
			
			# VERIFICAO QUAL E A CLASSIFICACAO CORRETA
			classeAudioAtual = self.verificarClassificacaoCorreta(nomeArquivo)
			
			# MONTO A MATRIZ DE FEATUREA DE CADA FRAME DO AUDIO ATUAL (a funcao abaixo
			# devolve uma matriz normal e faz o janelamento em outra funcao tb)
			matrizFeaturesAudioAtual = self.extrairFeaturesUnicoAudio(audioAtual, freqAmostragem, frameLength, overlapLength)
			
			# AGORA E HORA DE COLOCAR O NOME E A CLASSIFICACAO CORRETA NA MATRIZ
			# MAAAS, NA FUNCAO ABAIXO, O BICHO VIRA UM DATAFRAME PANDAS
			dataframeAudioAtual      = self.adicionarNomeArquivoEClasse(matrizFeaturesAudioAtual, nomeArquivo, classeAudioAtual)
			
			# CONCATENANDO O DATAFRAME DO AUDIO ATUAL AO DATAFRAME GERAL
			dataframeGeral = pd.concat([dataframeGeral, dataframeAudioAtual])
			
		# POR ULTIMO, RETORNO O DATAFRAME GERAL
		return dataframeGeral

	def escalonarFeatures(self, dataframeGeral):
		
		# Função para escalonar as features
		# Essa função recebe o dataframe geral, remove as primeira e última colunas (nome e classificação), 
		# faz o escalonamento e depois coloca as colunas de nome e classificação de volta.
	
		print("Escalonando features")
		
		# COPIANDO AS COLUNAS DE NOME E CLASSIFICACAO
		colunaArquivo       = dataframeGeral["nomeArquivo"]
		colunaClassificacao = dataframeGeral["classificacaoCorreta"]
		
		# DELETANDO AS COLUNAS ARQUIVO E CLASSIFICACAO
		dataframeGeral = dataframeGeral.drop(['nomeArquivo', 'classificacaoCorreta'], axis=1)
		
		# ESCALONANDO
		dataframeGeral = pd.DataFrame(StandardScaler().fit_transform(dataframeGeral))
		
		# COM ESSA COISA DE TIRA E POE COLUNA, O PANDAS NAO SABE LINDAR COM OS INDEXES,
		# ABAIXO EU ESTOU RESETANDO TUDO
		colunaArquivo.reset_index(inplace=True, drop=True)
		colunaClassificacao.reset_index(inplace=True, drop=True)
		dataframeGeral.reset_index(inplace=True, drop=True)
		
		# ADICIONANDO AS COLUNAS QUE FORAM EXCLUIDAS (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
		dataframeGeral.insert(0, "nomeArquivo", colunaArquivo, True)
		dataframeGeral.insert(len(dataframeGeral.columns), "classificacaoCorreta", colunaClassificacao, True)
		
		return dataframeGeral

	def reduzirDimensionalidade(self, dataframeGeral, nDimensoes):
	
		# Função que reduz a dimensionalidade do dataframe geral
		# Depois de escalonar, vamos reduzir a dimensionalidade utilizando PCA. 
		# A ideia é fazer com que o CSV já saia pronto para ser usado, logo, a redução 
		# de dimensionalidade deve vir nessa classe. Apesar disso, vou deixar o parâmetro 
		# nDimensoes lá no construtor da classe, ai caso ele seja nulo eu pulo a parte da 
		# redução de dimensionalidade e gero o CSV com todas as features.

		print("Reduzindo a dimensionalidade")
		
		# COPIANDO AS COLUNAS DE NOME E CLASSIFICACAO
		colunaArquivo       = dataframeGeral["nomeArquivo"]
		colunaClassificacao = dataframeGeral["classificacaoCorreta"]
		
		# DELETANDO AS COLUNAS ARQUIVO E CLASSIFICACAO
		dataframeGeral = dataframeGeral.drop(['nomeArquivo', 'classificacaoCorreta'], axis=1)
		
		# REDUZINDO A DIMENSIONALIDADE
		dataframeGeral = pd.DataFrame(PCA(n_components=nDimensoes).fit_transform(dataframeGeral))
		
		# COM ESSA COISA DE TIRA E POE COLUNA, O PANDAS NAO SABE LINDAR COM OS INDEXES,
		# ABAIXO EU ESTOU RESETANDO TUDO
		colunaArquivo.reset_index(inplace=True, drop=True)
		colunaClassificacao.reset_index(inplace=True, drop=True)
		dataframeGeral.reset_index(inplace=True, drop=True)
		
		# ADICIONANDO AS COLUNAS QUE FORAM EXCLUIDAS (posicaoNovaColuna, nomeNovaColuna, valorParaTodasAsLinhas)
		dataframeGeral.insert(0, "nomeArquivo", colunaArquivo, True)
		dataframeGeral.insert(len(dataframeGeral.columns), "classificacaoCorreta", colunaClassificacao, True)
		
		return dataframeGeral

	# DEFINICAO DAS FUNCOES QUE REALMENTE EXTRAEM AS FEATURES -----------------------------------------------

	def extrairRMS(self, sinal, frameLength, overlapLength):
		return librosa.feature.rms(y=sinal, frame_length=frameLength, hop_length=overlapLength)

	def extrairCentroideEspectral(self, sinal, freqAmostragem, frameLength, overlapLength):
		return librosa.feature.spectral_centroid(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

	def extrairLarguraBanda(self, sinal, freqAmostragem, frameLength, overlapLength):
		return librosa.feature.spectral_bandwidth(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

	def extrairPlanicidade(self, sinal, frameLength, overlapLength):
		return librosa.feature.spectral_flatness(y=sinal, n_fft=frameLength, hop_length=overlapLength)

	def extrairRolloff(self, sinal, freqAmostragem, frameLength, overlapLength):
		return librosa.feature.spectral_rolloff(y=sinal, sr= freqAmostragem, n_fft=frameLength, hop_length=overlapLength)

	def extrairZCR(self, sinal, frameLength, overlapLength):
		return librosa.feature.zero_crossing_rate(y=sinal, frame_length=frameLength, hop_length=overlapLength)

	def extrairMatrizMFCC(self, sinal, freqAmostragem):
		return librosa.feature.mfcc(y=sinal, sr=freqAmostragem)

	def extrairMFCCs(self, matrizMFCC):
		
		arrayMFCCs = []
		
		for linha in matrizMFCC:
			arrayMFCCs.append(np.mean(linha))
			
		return arrayMFCCs

	def extrairDeltas(self, matrizMFCC):
		matrizDelta = librosa.feature.delta(matrizMFCC, order=1)

		arrayDelta = []

		for linha in matrizDelta:
			arrayDelta.append(np.mean(linha))

		return arrayDelta

	def extrairDeltaDeltas(self, matrizMFCC):
		matrizDeltaDelta = librosa.feature.delta(matrizMFCC, order=2)

		arrayDeltaDelta = []

		for linha in matrizDeltaDelta:
			arrayDeltaDelta.append(np.mean(linha))

		return arrayDeltaDelta

	def extrairMelEspectrograma(self, sinal, freqAmostragem, frameLength, overlapLength):
		
		matrizMelEspectrograma = librosa.feature.melspectrogram(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
		
		arrayMelEspectrograma = []
		
		for coluna in matrizMelEspectrograma.T:
			arrayMelEspectrograma.append(np.mean(coluna))
		
		return arrayMelEspectrograma

	def extrairCromagramas(self, sinal, freqAmostragem, frameLength, overlapLength):
		
		matrizCromagramas = librosa.feature.chroma_stft(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
		
		arrayCromagramas = []
		
		for linha in matrizCromagramas:
			arrayCromagramas.append(np.mean(linha))
		
		return arrayCromagramas

	def extrairCromagramasQ(self, sinal, freqAmostragem):
		
		matrizCromagramasQ = librosa.feature.chroma_cqt(y=sinal, sr=freqAmostragem)
		
		arrayCromagramasQ = []
		
		for linha in matrizCromagramasQ:
			arrayCromagramasQ.append(np.mean(linha))
		
		return arrayCromagramasQ

	def extrairCromaCENSs(self, sinal, freqAmostragem):
		
		matrizCromaCENSs = librosa.feature.chroma_cens(y=sinal, sr=freqAmostragem)
		
		arrayCromaCENSs = []
		
		for linha in matrizCromaCENSs:
			arrayCromaCENSs.append(np.mean(linha))
		
		return arrayCromaCENSs

	def extrairContrastes(self, sinal, freqAmostragem, frameLength, overlapLength):
		
		matrizContrastes = librosa.feature.spectral_contrast(y=sinal, sr=freqAmostragem, n_fft=frameLength, hop_length=overlapLength)
		
		arrayConstrastes = []
		
		for linha in matrizContrastes:
			arrayConstrastes.append(np.mean(linha))
		
		return arrayConstrastes